In [ ]:
import pandas as pd
import re

In [ ]:
### funciones custom para extraer datos de columna categoria
# --- 1) Parser robusto: soporta \n, tabs, espacios múltiples, etc.
def parse_numbered(texto):
    if pd.isna(texto):
        return {}
    s = str(texto)
    # Captura "n: valor" hasta el siguiente "m:" o el final, de forma no codiciosa
    patron = re.compile(r'(\d+)\s*:\s*(.*?)(?=(?:\s*\d+\s*:)|\s*$)', flags=re.S)
    pares = patron.findall(s)
    out = {}
    for k, v in pares:
        # Normaliza espacios y limpia signos sueltos
        v = re.sub(r'\s+', ' ', v).strip(' ,.;:-')
        out[int(k)] = v
    return out

# --- 2) Construye (producto, categoría) por el mismo índice
def build_pairs(row):
    pdict = row['producto_dict']
    cdict = row['categoria_dict']
    # Solo índices que existan en ambas
    keys = sorted(set(pdict).intersection(cdict))
    return [(pdict[k], cdict[k]) for k in keys]

# --- 3) Función end-to-end para tu df
def explode_prod_cat(df, producto_col='producto', categoria_col='categoria', keep_cols=None):
    if keep_cols is None:
        # Conserva todo lo original excepto las 2 columnas que parseamos
        keep_cols = [c for c in df.columns if c not in {producto_col, categoria_col}]
    tmp = df.copy()

    tmp['producto_dict'] = tmp[producto_col].apply(parse_numbered)
    tmp['categoria_dict'] = tmp[categoria_col].apply(parse_numbered)

    tmp['pares'] = tmp.apply(build_pairs, axis=1)

    # Explode replica automáticamente el resto de columnas (p.ej. sentimiento)
    tmp = tmp.explode('pares', ignore_index=True)
    tmp = tmp[tmp['pares'].notna()].copy()

    tmp[['producto_final','categoria_final']] = pd.DataFrame(
        tmp['pares'].tolist(), index=tmp.index
    )

    cols = keep_cols + ['producto_final', 'categoria_final']
    return tmp[cols]

# --- 4) (Opcional) Reporte de calidad: detecta si quedó algún "n:" sin limpiar
def quality_check(df_long):
    mask = (
        df_long['producto_final'].astype(str).str.contains(r'\b\d+\s*:', na=False) |
        df_long['categoria_final'].astype(str).str.contains(r'\b\d+\s*:', na=False)
    )
    return df_long[mask]


#separar cat y subcat
def categoria_subcategoria(texto):
    a = texto.split(":")
    categoria = a[0]
    subcategoria = a[1]

    return categoria, subcategoria



In [ ]:
df = pd.read_parquet("./data/Silver/Cleaned_data_features_productos_categorias.parquet")
df_limpieza = df[df.producto != 'No aplica']
df_limpieza

In [ ]:
#sacamos una linea por objeto de la lista con explode. separamos 1: productoi_1 2:producto_2 uno por linea. 

df_long = explode_prod_cat(
    df_limpieza,
    producto_col='producto',
    categoria_col='categoria',
    keep_cols=['User','tweets','search','fecha_captura','sentimiento','imagen_marca','sentimiento_producto','comparativa_producto']
)

df_long

In [ ]:
# Filas problemáticas (si quedara algo por limpiar)
df_problema = quality_check(df_long)
df_problema

In [ ]:
#quitamos las categorias no detectadas
df_long = df_long[df_long.categoria_final != "Indeterminado : Indeterminado"]
df_long

In [ ]:
sin_sub = df_long[~df_long["categoria_final"].str.contains(":", na=False)]
print((len(sin_sub)))
sin_sub

In [ ]:
#limpiamos esos valores aberrantes y malas salidas del modelo. 

index = [16217, 32350, 37803, 47454, 60822, 61744, 78215, 90759, 102201,
         103158, 104897, 116565, 121342, 126711, 127640]

cat_subcat = [
    'Huevos, leche y mantequilla : Huevos',
    "Aceite, especias y salsas : Otras salsas", 
    'Cacao, café e infusiones : Café molido y en grano',
    'Cereales y galletas : Galletas',
    'Agua y refrescos : Isotónico y energético',
    'Panadería y pastelería : Pan de horno',
    'Congelados : Rebozados',
    'Bebé : Higiene y cuidado',
    'Panadería y pastelería : Bollería de horno',
    'Cacao, café e infusiones : Té e infusiones',
    'Indeterminado : Indeterminado',
    'Bodega : Cerveza',
    'Indeterminado : Indeterminado',
    'Postres y yogures : Yogures líquidos',
    'Indeterminado : Indeterminado'
]

# asignar los nuevos valores en la columna "categoria_final"
df_long.loc[index, "categoria_final"] = cat_subcat

In [ ]:
### separamos categorias y subcategorias

df_long[["categoria", "subcategoria"]] = df_long["categoria_final"].apply(categoria_subcategoria).apply(pd.Series)
df_long

In [ ]:
#tenemos que limpiar las categorias que no han tenido buena salida
for i in df_long.categoria.value_counts().items():
  print(i)

In [ ]:
categorias_validas = [
    "Aceite, especias y salsas",
    "Agua y refrescos",
    "Aperitivos",
    "Arroz, legumbres y pasta",
    "Azúcar, caramelos y chocolate",
    "Bebé",
    "Bodega",
    "Cacao, café e infusiones",
    "Carne",
    "Cereales y galletas",
    "Charcutería y quesos",
    "Congelados",
    "Conservas, caldos y cremas",
    "Cuidado del cabello",
    "Cuidado facial y corporal",
    "Fitoterapia y parafarmacia",
    "Fruta y verdura",
    "Huevos, leche y mantequilla",
    "Limpieza y hogar",
    "Maquillaje",
    "Marisco y pescado",
    "Mascotas",
    "Panadería y pastelería",
    "Pizzas y platos preparados",
    "Postres y yogures",
    "Zumos"
]

map_categorias = {
    "Panadería": "Panadería y pastelería",
    "Pasta": "Arroz, legumbres y pasta",
    "Frutas": "Fruta y verdura",
    "Fruta": "Fruta y verdura",
    "Verduras": "Fruta y verdura",
    "Hortalizas": "Fruta y verdura",
    "Frutas y verduras": "Fruta y verdura",
    "Legumbres": "Arroz, legumbres y pasta",
    "Legumbres y cereales": "Arroz, legumbres y pasta",
    "Cereales": "Cereales y galletas",
    "Carnes": "Carne",
    "Bebidas": "Agua y refrescos",
    "Snacks": "Aperitivos",
    "Snacks y aperitivos": "Aperitivos",
    "Dulces": "Azúcar, caramelos y chocolate",
    "Dulces y snacks": "Azúcar, caramelos y chocolate",
    "Postres y dulces": "Postres y yogures",
    "Helados": "Congelados",
    "Lácteos": "Huevos, leche y mantequilla",
    "Lactarios": "Huevos, leche y mantequilla",
    "Lactaria": "Huevos, leche y mantequilla",
    "Lactantes": "Bebé",
    "Cuidado personal": "Cuidado facial y corporal",
    "Cuidado del hogar": "Limpieza y hogar",
    "Higiene": "Limpieza y hogar",
    "Comidas para llevar": "Pizzas y platos preparados"
}

def limpiar_categoria(cat):
    cat = cat.strip()  # quitar espacios
    if cat in categorias_validas:
        return cat
    elif cat in map_categorias:
        return map_categorias[cat]
    else:
        return None  # marcar para eliminar

In [ ]:
df_long["categoria"] = df_long["categoria"].apply(limpiar_categoria)
df_long.isna().sum()

In [ ]:
df_long = df_long.dropna(subset=["categoria"])

In [ ]:
df_long.isna().sum()

In [ ]:
df_to_Gold = df_long[['fecha_captura', 'sentimiento', 'categoria', 'subcategoria']]
df_to_Gold

In [ ]:
df_to_Gold['fecha_captura'] = pd.to_datetime(df_to_Gold['fecha_captura'])
df_to_Gold.info()

In [ ]:
##### limpieza de algunos datos extraños 
subcategorias_validas = {
    "Accesorios de pelo": "Indeterminado",
    "Aceite, especias y salsas": "Aceite, vinagre y sal",
    "Aceitunas y encurtidos": "Aceitunas y encurtidos",
    "Acondicionador": "Indeterminado",
    "Acondicionadores": "Acondicionador y mascarilla",
    "Agua con gas": "Agua",
    "Aguardiente": "Cerveza",
    "Alcachofas": "verdura",
    "Alcohol": "Licores",
    "Alubias": "Legumbres",
    "Alubias riojana": "Legumbres",
    "Anís": "Licores",
    "Aperitivos salados": "Patatas fritas y snacks",
    "Avena": "Leche y bebidas vegetales",
    "BB  Cream": "Bases de maquillaje y corrector",
    "BB  cream": "Bases de maquillaje y corrector",
    "Base": "Bases de maquillaje y corrector",
    "Baño": "Gel y jabón de manos",
    "Bebidas": "Licores",
    "Bebidas alcohólicas": "Licores",
    "Bollería": "Bollería de horno",
    "Bolsas para el pan": "Indeterminado",
    "Cabello": "Indeterminado",
    "Cacao": "Cacao soluble y chocolate a la taza",
    "Cafetera": "Café molido y en grano",
    "Café": "Café cápsula y monodosis",
    "Café instantáneo": "Café soluble y otras bebidas",
    "Caquis": "Fruta",
    "Carne": "Cerdo",
    "Carne congelada": "Carne congelada",
    "Carne fresca": "Arreglos",
    "Cepillos": "Indeterminado",
    "Cepillos y peines": "Indeterminado",
    "Cerveza y sidra": "Cerveza",
    "Champiñones": "Verdura",
    "Champú": "Champú",
    "Chocolates y bombones": "Chocolate",
    "Chopped  y mortadela": "Chopped y mortadela",
    "Chuches": "Golosinas",
    "Chuleta": "Cerdo",
    "Chuletones del  mercadona": "Vacuno",
    "Comidas para llevar": "Listo para Comer",
    "Comino": "Especias",
    "Congelados": "Pescado congelado",
    "Conservas": "Indeterminado",
    "Conservas de verduras": "Conservas de verdura y frutas",
    "Contenedores": "Utensilios de limpieza y calzado",
    "Cordero": "Conejo y cordero",
    "Costillas y chuletas": "Cerdo",
    "Cous   cous": "Arroz",
    "Couscous": "Arroz",
    "Cous cous" : "Arroz",
    "Crema de leche": "Indeterminado",
    "Crema facial": "Cuidado e higiene facial",
    "Cremas": "Cuidado corporal",
    "Cremas Deliplus": "Cuidado corporal",
    "Croasancitos": "Bollería de horno",
    "Croissants": "Bollería de horno",
    "Croquetas": "Rebozados",
    "Cruasanes": "Bollería de horno",
    "Cuidado de la piel": "Cuidado corporal",
    "Cuidado facial": "Cuidado e higiene facial",
    "Desatascadores": "Lejía y líquidos fuertes",
    "Desayunos": "Indeterminado",
    "Dulces y chuches": "Bollería envasada",
    "Duraznos": "Fruta",
    "Embuti": "Embutido",
    "Embutido": "Embutido",
    "Embutido curado": "Embutido curado",
    "Embutidos": "Embutido",
    "Empanadas": "Bollería de horno",
    "Empanadillas": "Bollería de horno",
    "Empanadillas y croquetas": "Bollería de horno",
    "Enlatados": "Indeterminado",
    "Entrana": "Arreglos",
    "Espumosos": "Vino lambrusco y espumoso",
    "Fajitas": "Verdura",
    "Fajitas de sobre": "Indeterminado",
    "Fruta seca": "Fruta",
    "Frutas": "Fruta",
    "Frutas de hueso": "Indeterminado",
    "Frutas y verduras": "Fruta",
    "Frutos rojos": "Té e infusiones",
    "Gambas": "Marisco",
    "Gel de ducha": "Gel y jabón de manos",
    "Gel de manos": "Gel y jabón de manos",
    "Ginebra": "Licores",
    "Golosinas": "Golosinas",
    "Helados 1": "Helados",
    "Hielo": "Hielo",
    "Higiene personal": "Indeterminado",
    "Hortalizas": "Verdura",
    "Hortalizas frescas": "Verdura",
    "Infusiones": "Té e infusiones",
    "Jamón y lomo": "cerdo",
    "Jerez": "Licores",
    "Judías de La Granja de Mercadona": "Indeterminado",
    "Labios": "Labios",
    "Lavado": "Limpiahogar y friegasuelos",
    "Lavado y limpieza": "Limpiahogar y friegasuelos",
    "Leche": "Leche y bebidas vegetales",
    "Leche sin lactosa": "Leche y bebidas vegetales",
    "Leche vegetal": "Leche y bebidas vegetales",
    "Leche y nata": "Leche y bebidas vegetales",
    "Legumbres enlatadas": "Legumbres",
    "Lentejas": "Legumbres",
    "Lentejas enlatadas": "Legumbres",
    "Licor": "Licores",
    "Licores 1": "Licores",
    "Limpiahogar  y friegasuelos": "Limpiahogar y friegasuelos",
    "Limpieza baño y WC": "Limpieza baño y WC",
    "Limpieza cocina": "Limpieza cocina",
    "Limpieza del hogar": "Limpiahogar y friegasuelos",
    "Limpieza facial": "Cuidado e higiene facial",
    "Limpieza general": "Limpiahogar y friegasuelos",
    "Listo para Comer": "Listo para Comer",
    "Macarrones": "Pasta y fideos",
    "Manicura": "Manicura y pedicura",
    "Mantecados y turrones": "Dulces navideños",
    "Manzanas": "Fruta",
    "Masas": "Harina y preparado repostería",
    "Mascarillas faciales": "Cuidado e higiene facial",
    "Mayonesa,  ketchup  y mostaza": "Mayonesa, ketchup y mostaza",
    "Mayonesa,  ketchup  y mostaza 8": "Mayonesa, ketchup y mostaza",
    "Melones y sandías": "Fruta",
    "Máquinas de café": "Indeterminado",
    "Naranjas": "Naranja",
    "Nata": "Mantequilla y margarina",
    "Nata líquida": " Mantequilla y margarina",
    "Nata para montar": "Mantequilla y margarina",
    "Otras  sals": "Otras salsas",
    "Paella": "Indeterminado",
    "Paleta Low  Cost": "Indeterminado",
    "Pan": "Pan de horno",
    "Pan de Navidad": "Dulces navideños",
    "Pan de espelta": "Pan de molde y otras especialidades",
    "Pan de espelta y pasta integral": "Pan de molde y otras especialidades",
    "Pan  recien  hecho": "Pan de horno",
    "Panadería": "Harina y preparado repostería",
    "Panadería y pastelería": "Bollería de horno",
    "Panecillos": "Pan de horno",
    "Panecillos con pasas": "Indeterminado",
    "Panes y bollería": "Bollería de horno",
    "Papel higiénico y celulosa": "Papel higiénico y celulosa",
    "Pasas y frutos secos": "Indeterminado",
    "Pasta": "Pasta y fideos",
    "Pasta fresca": "Pasta y fideos",
    "Pasta precocinada": "Pasta y fideos",
    "Pasta rellena": "Pasta y fideos",
    "Pasta seca": "Pasta y fideos",
    "Pasticho": "Indeterminado",
    "Patatas": "Patatas fritas y snacks",
    "Pañales": "Toallitas y pañales",
    "Pañuelos": "Papel higiénico y celulosa",
    "Peines": "Indeterminado",
    "Pelo": "Indeterminado",
    "Peluquería": "Indeterminado",
    "Peras y ciruelas": "Fruta",
    "Pescado": "Pescado fresco",
    "Pescado precocinado": "Pescado congelado",
    "Picados": "Arreglos",
    "Pienso": "Perro",
    "Plátano": "Fruta",
    "Plátanos": "Fruta",
    "Pollo": "Aves y pollo",
    "Polvorones": "Dulces navidad",
    "Polvorones veganos": "Dulces navidad",
    "Porras": "Tartas y churros",
    "Pringles": "Patatas fritas y snacks",
    "Protector solar y  aftersun": "Protector solar y aftersun",
    "Purés": "Alimentación infantil",
    "Queso": "Queso curado, semicurado y tierno",
    "Quesos": "Queso curado, semicurado y tierno",
    "Quiches": "Indeterminado",
    "Rebozados": "Rebozados",
    "Refresco de manzana": "Indeterminado",
    "Refresco de sandía hacendado": "Indeterminado",
    "Refrescos": "Refresco de cola",
    "Ron": "Licores",
    "Rosca de jamón y queso": "Indeterminado",
    "Roscones": "Dulces navideños",
    "Rostro": "Indeterminado",
    "Rímel": "Ojos",
    "Sandias": "Fruta",
    "Sandía": "Fruta",
    "Sandías": "Fruta",
    "Secreto de cerdo": "Cerdo",
    "Snacks salados": "Patatas fritas y snacks",
    "Sándwiches": "Indeterminado",
    "Ternera": "Vacuno",
    "Tinte o mechas": "Coloración cabello",
    "Turrones": "Dulces navideños",
    "Turrones y polvorones": "Dulces navideños",
    "Té": "Té e infusiones",
    "Té e infusiones": "Té e infusiones",
    "Uñas": "Manicura y pedicura",
    "Verduras": "Verdura",
    "Verduras al wok": "Verdura",
    "Verduras congeladas": "Verdura",
    "Verduras y hortalizas": "Verdura",
    "Vermut": "Licores",
    "Vino": "Vino blanco",
    "Vino dulce": "Vino Blanco",
    "Vino espumoso": "Sidra y cava",
    "Vodka": "Licores",
    "Yogomix  de yogur azucarado y copos de cereales chocolateados": "Gelatina y otros postres",
    "Yogures": "Yogures naturales y sabores",
    "Yogures con fruta": "Yogures naturales y sabores",
    "Yogures de frutas": "Yogures naturales y sabores",
    "Yogures de soja": "Postres de soja",
    "Yogures sin lactosa": "Yogures naturales y sabores",
    "Zumos": "Fruta variada",
    "licores": "Licores",
    "ñoquis": "Pasta y fideos"
}

len(subcategorias_validas)


In [ ]:
def limpiar_subcategoria(subcat: str) -> str:
    """Limpia y normaliza una subcategoría según el diccionario y las válidas del JSON"""
    subcat = subcat.strip()
    # aplicar normalización
    subcat = subcategorias_validas.get(subcat, subcat)
    # si no es válida, mandamos a Indeterminado
#    if subcat not in subcategorias_validas:
#        return "Indeterminado"
    return subcat

In [ ]:
df_to_Gold["Subcategoria_limpia"] = df_to_Gold["subcategoria"].apply(limpiar_subcategoria)
df_to_Gold

In [ ]:
len(np.unique(df_to_Gold.subcategoria))

In [ ]:
import numpy as np
len(np.unique(df_to_Gold.Subcategoria_limpia))

In [ ]:
pd.DataFrame((df_to_Gold[df_to_Gold['subcategoria'].str.strip() != df_to_Gold['Subcategoria_limpia']]).value_counts())

In [ ]:
pd.DataFrame((df_to_Gold[df_to_Gold['subcategoria'].str.strip() != df_to_Gold['Subcategoria_limpia']]).value_counts()).groupby(by='Subcategoria_limpia').count()

In [ ]:
indice_unas = df_to_Gold[df_to_Gold['subcategoria'].str.strip() == 'Uñas'].index

for i in indice_unas:
  df_to_Gold.at[i,'categoria'] = 'Cuidado facial y corporal'

df_to_Gold[df_to_Gold['subcategoria'].str.strip() == 'Uñas']

In [ ]:
df_to_Gold[df_to_Gold['subcategoria'].str.strip() == 'Fajitas']


In [ ]:
df_to_Gold.drop(31799, inplace=True)
df_to_Gold[df_to_Gold['subcategoria'].str.strip() == 'Fajitas']

In [ ]:
df_to_Gold['subcategoria'] = df_to_Gold['Subcategoria_limpia']
df_to_Gold.columns

In [ ]:
df_to_Gold = df_to_Gold[['fecha_captura', 'sentimiento', 'categoria', 'subcategoria']]
df_to_Gold

In [ ]:
df_to_Gold.to_parquet("./data/Gold/Gold_Categorias.parquet", index=False)